In [ ]:
#Imports and Setup

import os
import pandas as pd
import numpy as np
import sys
sys.path.append('../scripts')
from utils import make_project_dirs, save_histplot, save_boxplot, save_figure


# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt
import seaborn as sns

# Ensure outputs/figures folders exist
make_project_dirs()

ModuleNotFoundError: No module named 'scripts.utils'

In [ ]:
#Load Data

DATA_PATH = '../data/pollution_us_2000_2016.csv'   # Adjust path if running from project root

df = pd.read_csv(DATA_PATH)
df.head()

In [ ]:
#EDA - Structure & Missing Values

print(df.info())
print(df.describe())
print(df.isnull().sum())

# Optional: Save missing values as CSV for report
df.isnull().sum().to_csv('outputs/missing_values_summary.csv')

In [ ]:
#Data Cleaning

# Drop columns with excessive missing data, or fill if appropriate
df = df.dropna(subset=["Ozone Mean", "CO Mean", "NO2 Mean", "SO2 Mean", "PM2.5 Mean"])
df = df.copy()  # Prevent SettingWithCopyWarning

# Convert 'Date Local' to datetime
df['Date Local'] = pd.to_datetime(df['Date Local'])

# Save cleaned sample for report
df.sample(100).to_csv('outputs/cleaned_sample.csv', index=False)

In [ ]:
#EDA Visualizations

save_histplot(df["Ozone Mean"], "Ozone Mean Distribution", "ozone_mean_distribution")
save_histplot(df["PM2.5 Mean"], "PM2.5 Mean Distribution", "pm25_mean_distribution")
save_boxplot(df["Ozone Mean"], "Ozone Mean Boxplot", "ozone_mean_boxplot")
save_boxplot(df["PM2.5 Mean"], "PM2.5 Mean Boxplot", "pm25_mean_boxplot")

# Show correlation heatmap in notebook (save manually if desired)
plt.figure(figsize=(8,6))
sns.heatmap(df[["Ozone Mean", "CO Mean", "NO2 Mean", "SO2 Mean", "PM2.5 Mean"]].corr(), annot=True, cmap="BuGn")
plt.title("Feature Correlation Heatmap")
plt.tight_layout()
plt.savefig("outputs/figures/eda/feature_correlation_heatmap.png")
plt.show()

In [ ]:
#Feature/Target Engineering

# Example: Classify if Ozone Mean > EPA limit (0.070 ppm)
df['Ozone_High'] = (df['Ozone Mean'] > 0.070).astype(int)

features = ["CO Mean", "NO2 Mean", "SO2 Mean", "PM2.5 Mean"]
target = "Ozone_High"

In [ ]:
#Split & Scale

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Train and Evaluate Classifier

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# ROC/AUC
y_proba = clf.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, y_proba)
print("ROC AUC:", roc_auc)

# ROC Curve
fig, ax = plt.subplots()
RocCurveDisplay.from_estimator(clf, X_test, y_test, ax=ax)
save_figure(fig, "model_eval", "random_forest_roc_curve")

In [ ]:
#Cross-validation

cv_scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print("Cross-validated AUC scores:", cv_scores)
print("Mean CV AUC:", np.mean(cv_scores))

In [ ]:
#Feature Importances

importances = clf.feature_importances_
feat_df = pd.DataFrame({"Feature": features, "Importance": importances}).sort_values("Importance", ascending=False)

plt.figure(figsize=(7,4))
sns.barplot(x="Importance", y="Feature", data=feat_df, palette="BuGn_d")
plt.title("Random Forest Feature Importances")
plt.tight_layout()
plt.savefig("outputs/figures/model_eval/feature_importance.png")
plt.show()